# Spark Streaming

# pré requis

Vous devez d’abord exécuter Netcat (un petit utilitaire présent dans la plupart des systèmes de type Unix) en tant que serveur de données à l’aide de

<code>$ nc -lk 9999</code>

<h2> après avoir executer toute les cellule copiez le texte que vous voulez dans la terminal netct, spark va lire les données et afficher le résultat </h2>

Premièrement, nous importons les noms des classes Spark Streaming et certaines conversions implicites de StreamingContext dans notre environnement afin d’ajouter des méthodes utiles aux autres classes dont nous avons besoin (comme DStream). StreamingContext est le point d'entrée principal pour toutes les fonctionnalités de diffusion en continu. Nous créons un StreamingContext local avec deux threads d'exécution et un intervalle de traitement par lots d'une seconde.<br>
<b> c'est un streamingcontext qui cherche les nouvelle données toute les secondes </b>

In [ ]:
import org.apache.spark._
import org.apache.spark.streaming._
import org.apache.spark.streaming.StreamingContext._ // not necessary since Spark 1.3

// Create a local StreamingContext with two working thread and batch interval of 1 second.
// The master requires 2 cores to prevent a starvation scenario.

val conf = new SparkConf().setMaster("local[2]").setAppName("NetworkWordCount")
val ssc = new StreamingContext(conf, Seconds(10))

En utilisant ce contexte, nous pouvons créer un DStream qui représente les données en continu d'une source TCP, spécifiées comme nom d'hôte (par exemple localhost) et port (par exemple 9999).

In [ ]:
// Create a DStream that will connect to hostname:port, like localhost:9999
val lines = ssc.socketTextStream("localhost", 9999)

Cette ligne DStream représente le flux de données qui sera reçu du serveur de données. Chaque enregistrement de ce DStream est une ligne de texte. Ensuite, nous voulons diviser les lignes par des caractères d'espace.

In [ ]:
// Split each line into words
val words = lines.flatMap(_.split(" "))

flatMap est une opération DStream d'un à plusieurs qui crée un nouveau DStream en générant plusieurs nouveaux enregistrements à partir de chaque enregistrement du DStream source. Dans ce cas, chaque ligne sera scindée en plusieurs mots et le flux de mots est représenté par les mots DStream. Ensuite, nous voulons compter ces mots.

In [ ]:
val pairs = words.map(word => (word, 1))
val wordCounts = pairs.reduceByKey(_ + _)

// Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.print()

Les mots DStream sont en outre mappés (transformation un à un) en un flux DStream de (mot, 1) paires, qui est ensuite réduit pour obtenir la fréquence des mots dans chaque lot de données. Enfin, wordCounts.print () imprimera quelques-uns des comptes générés chaque seconde.

Notez que lorsque ces lignes sont exécutées, Spark Streaming configure uniquement le calcul qu'il effectuera au démarrage et aucun traitement réel n'a encore commencé. Pour commencer le traitement après que toutes les transformations ont été configurées, nous appelons finalement

In [ ]:
ssc.start()             // Start the computation
ssc.awaitTermination()  // Wait for the computation to terminate